In [6]:
## 确认同源基因bias是否是可表达bias基因对
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/homo_gene2/gene_bias/all_tissue_bias_gene_pair.bed", sep='\t')
df_exp = df.query("TPM_At/TPM_Dt>=2 | TPM_Dt/TPM_At>=2")
df_exp = df_exp.query("TPM_At>=0.5 | TPM_Dt>=0.5")
df = pd.DataFrame(df_exp[['tissues', 'sig']].value_counts(), columns=['number'])
df.reset_index(inplace=True)
df = df.sort_values(['tissues', 'sig'])
print(df)
print(df.shape[0], df_exp.shape[0])

        tissues    sig  number
7        anther  BiasA    3107
4        anther  BiasD    3264
15    cotyledon  BiasA    2757
10    cotyledon  BiasD    2938
8   fiber_10DPA  BiasA    3026
5   fiber_10DPA  BiasD    3259
9   fiber_20DPA  BiasA    2953
6   fiber_20DPA  BiasD    3110
17    hypocotyl  BiasA    2666
14    hypocotyl  BiasD    2799
21         leaf  BiasA    2521
12         leaf  BiasD    2849
23        ovule  BiasA    2256
19        ovule  BiasD    2541
3         petal  BiasA    3399
1         petal  BiasD    3542
22      radicle  BiasA    2449
20      radicle  BiasD    2538
18         root  BiasA    2609
13         root  BiasD    2808
16         stem  BiasA    2719
11         stem  BiasD    2892
2        stigma  BiasA    3424
0        stigma  BiasD    3622
24 70048


In [10]:
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/DEG/tissue/J668_mutipTissues_DEG.csv")
df_anther_ovule = df.query("tags=='anther-ovule'")
df_anther_ovule['sig'].value_counts()

Not     34264
Down    17016
Up      15496
Name: sig, dtype: int64

In [15]:
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/DEG/tissue/J668_mutipTissues_DEG.csv")
df[['control', 'treatment']] = df['tags'].str.split('-', expand=True)
df.rename(columns={"sig":'sigs'}, inplace=True)
df.drop('tags', inplace=True, axis=1)
df.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/DEG/tissue/J668_mutipTissues_DEG.csv", index=False)

In [3]:
## 合并所有组织的ATAC peaks
import pandas as pd
tissues = ['anther', 'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'ovule', 'petal', 'radicle', 'stigma']
df = pd.DataFrame()
for t in tissues:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/ATAC/rawdata/{}_total_peaks_merge_BL.bed".format(t)
    df_tmp = pd.read_csv(input_f, sep="\t", names =['chorm', 'start', 'end'])
    df_tmp['tissues'] = [t]*df_tmp.shape[0]
    df = pd.concat([df, df_tmp], axis=0)
df.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/rawdata/J668_total_peaks_merge_BL.csv", index=False)

In [7]:
## 合并所有组织的 H3K27ac
import pandas as pd
tissues = ['cotyledon', 'hypocotyl', 'ovule', 'radicle', 'root', 'stem', 'stigma']
df = pd.DataFrame()
for t in tissues:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/peaks/{}_H3K27ac_remvBL.bed".format(t)
    df_tmp = pd.read_csv(input_f,  sep="\t", names=['chrom', 'start', 'end'])
    df_tmp['tissues'] = [t]*df_tmp.shape[0]
    df = pd.concat([df, df_tmp], axis=0)
df.sort_values(by=['tissues', 'chrom', 'start'], inplace=True)
df.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/peaks/J668_H3K27ac_remvBL.csv", index=False)

In [6]:
## 合并所有组织的 H3K4me3
import pandas as pd
tissues = ['anther', 'cotyledon', 'hypocotyl', 'leaf', 'ovule', 'petal', 'radicle', 'stem', 'stigma']
df = pd.DataFrame()
for t in tissues:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/peaks/{}_H3K4me3_remvBL.bed".format(t)
    df_tmp = pd.read_csv(input_f,  sep="\t", names=['chrom', 'start', 'end'])
    df_tmp['tissues'] = [t]*df_tmp.shape[0]
    df = pd.concat([df, df_tmp], axis=0)
df.sort_values(by=['tissues', 'chrom', 'start'], inplace=True)
df.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/peaks/J668_H3K4me3_remvBL.csv", index=False)


In [25]:
## 合并所有组织多种修饰peaks所注释的基因
import pandas as pd
def peaks_geneid(tags):
    df = pd.DataFrame()
    if tags=='ATAC':
        tissues = ['anther', 'fiber_10DPA', 'fiber_20DPA', 'hypocotyl', 'ovule', 'petal', 'radicle', 'stigma']
        input_dir = "G:/Billfish/J668_multip_tissue_3D-genome/ATAC/rawdata"
    elif tags=='H3K27ac':
        tissues = ['cotyledon', 'hypocotyl', 'ovule', 'radicle', 'root', 'stem', 'stigma']
        input_dir = "G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/peaks/annotation"
    else:
        tissues = ['anther', 'cotyledon', 'hypocotyl', 'leaf', 'ovule', 'petal', 'radicle', 'stem', 'stigma']
        input_dir = "G:/Billfish/J668_multip_tissue_3D-genome/Chip_Seq/peaks/annotation"
    for t in tissues:
        if tags == 'ATAC':
            input_f = "{0}/{1}_peaks_annotation.bed".format(input_dir, t)
        else:
            input_f = "{0}/{1}_{2}_peaks_annotation.bed".format(input_dir, t, tags)
        df_tmp = pd.read_csv(input_f, sep=" ")
        df = pd.concat([df, df_tmp], axis=0)
    df_return = df[['geneId']].copy()
    df_return.columns = [tags]
    df_return.drop_duplicates(keep='first', inplace=True)
    df_return.reset_index(drop=True, inplace=True)
    return df_return
df_ATAC = peaks_geneid('ATAC')
df_H3K27ac = peaks_geneid("H3K27ac")
df_H3K4me3 = peaks_geneid("H3K4me3")
print(df_ATAC.shape[0], df_H3K27ac.shape[0], df_H3K4me3.shape[0])
df_return = pd.concat([df_ATAC, df_H3K27ac, df_H3K4me3], axis=1, ignore_index=True)
df_return.columns = ['ATAC', 'H3K27ac', 'H3K4me3']
df_return.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC_ChIP/J668_ATAC_H3K27ac_H3K4me3_peaks_annotation_gene.csv", index=False)


20755 50701 59528


In [ ]:
import pandas as pd
df = pd.read_csv("")

第1段长度: 0.9303286876159645 厘米
第2段长度: 1.558789428299053 厘米
第3段长度: 0.7335070276064921 厘米
第4段长度: 2.7130258354204657 厘米
第5段长度: 0.4215133175786807 厘米
第6段长度: 3.1788357034793435 厘米
